In [9]:
import numpy as np
import pandas as pd

In [10]:
#loading the dataset
df=pd.read_csv('Clean_Data.csv')
df.head()

,Is Captain(1=yes),Is Wktkeeper(1=Yes),Nationality(1=Overseas),Name,Team,IPL 4 Franchise,Player cost USD,matches,innings played,no,...,half_centuries_ipl,fours_ipl,sixes_ipl,matches_bowl_ipl,balls_bowled_ipl,total_runs_given_ipl,wickets_taken_ipl,bowling_avg_ipl,economy_ipl,3wickets_ipl
0,0.0,0.0,1,Aiden Blizzard,Australia,Mumbai Indians,20000.0,40.0,40.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1,Jonathan Vandiar,South Africa,Royal Challengers Bangalore,20000.0,18.0,18.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1,Nathan Rimmington,Australia,Kings XI Punjab,20000.0,23.0,8.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1,Rilee Rossouw,South Africa,Royal Challengers Bangalore,20000.0,20.0,20.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1,Robert Frylinck,South Africa,Delhi Daredevils,20000.0,25.0,22.0,8.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
#Encoding the variables
data = pd.get_dummies(df, columns=['Team', 'IPL 4 Franchise'])
data.shape


(129, 69)

In [12]:
#Plotting data with 5 variables
import plotly.express as px
fig2 = px.scatter_3d(df, x="Player cost USD", y="matches",z="innings played",
                     color="Strike rate",size="Catches per match")
fig2.update_layout(title="5 Features Representation")
fig2.show()

In [13]:
#Training data
X=data.drop(['Name','Is bowler?','Is batsman'],axis=1)
#data=data.drop(['Name','Is bowler?','Is batsman'],axis=1)

In [14]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go

#Plotting the graph to determine optimal number of clusters
scaler = MinMaxScaler()
scaler.fit(X)
X=scaler.transform(X) #Scaling the data
#INERTIA: Mean square distance from centroid to other points
inertia = []
#take clusters range(1,11) and fit data for every cluster value
for i in range(1,11):
    kmeans = KMeans(
        n_clusters=i, init="k-means++",
        n_init=10,
        tol=1e-04, random_state=42
    )
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)
fig = go.Figure(data=go.Scatter(x=np.arange(1,11),y=inertia))
fig.update_layout(title="Inertia vs Cluster Number",xaxis=dict(range=[0,11],title="Cluster Number"),
                  yaxis={'title':'Inertia'},
                 annotations=[
        dict(
            x=3,
            y=inertia[2],
            xref="x",
            yref="y",
            text="Elbow!",
            showarrow=True,
            arrowhead=7,
            ax=20,
            ay=-40
        )
    ])

In [18]:
kmeans = KMeans(
        n_clusters=3, init="k-means++",
        n_init=10,
         random_state=42
    )
kmeans.fit(X)
clusters=pd.DataFrame(X,columns=data.drop(['Name','Is bowler?','Is batsman'],axis=1).columns)
clusters['label']=kmeans.labels_
#print(clusters)
polar=clusters.groupby("label").mean().reset_index()
#print(polar)
polar=pd.melt(polar,id_vars=["label"])
fig4 = px.line_polar(polar, r="value", theta="variable", color="label", line_close=True,height=800,width=1400)
fig.show()

   label  Is Captain(1=yes)  Is Wktkeeper(1=Yes)  Nationality(1=Overseas)  \
0      0           0.195122             0.268293                 0.512195   
1      1           0.054054             0.000000                 0.405405   
2      2           0.000000             0.078431                 0.882353   

   Player cost USD   matches  innings played        no  runs scored  \
0         0.408844  0.501470        0.528840  0.278049     0.523454   
1         0.260220  0.382787        0.205077  0.260661     0.091580   
2         0.093755  0.241413        0.235559  0.180392     0.167293   

   highest score  ...  IPL 4 Franchise_Chennai Super Kings  \
0       0.605378  ...                             0.121951   
1       0.194040  ...                             0.162162   
2       0.365259  ...                             0.137255   

   IPL 4 Franchise_Deccan Chargers  IPL 4 Franchise_Delhi Daredevils  \
0                         0.146341                          0.073171   
1            

In [14]:
pie=clusters.groupby('label').size().reset_index()
pie.columns=['label','value']
px.pie(pie,values='value',names='label',color=['blue','red','green'])